In [1]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [3]:
url = 'https://en.bidfax.info/'

In [4]:
driver.get(url)

#### After calling select_make() function it will ask you to input a make of a car and then it will select it. If that make will not exit it will tell you to put the existing one.

In [5]:
def select_make():
    dropdown = driver.find_elements(By.XPATH, '//span[@class = "drop-down"]' )
    make = str(input('Please write the make you want to select:  '))
    if make == '':
        print(f'{make} make does not exist in this list. Please try again.')
        return select_make()
    else:
        def search_for_make(make):
            try:
                dropdown[0].click()
            except:
                pass
            try:
                make_ = driver.find_element(By.LINK_TEXT, make)
                make_.click()
                print(f"{make} make is selected successfully!")
            except:
                print(f'{make} make does not exist in this list. Please try again.')
                return select_make()
        search_for_make(make)
        time.sleep(1)

#### After calling select_model() fucntion it will ask you to input a model of a car and then it will select it. If that make will not exit it will tell you to put the existing one.

In [6]:
def select_model():
    dropdown = driver.find_elements(By.XPATH, '//span[@class = "drop-down"]' )
    model = str(input('Please write the model you want to select:  '))
    if model == '':
        print(f'{model} model does not exist in this list. Please try again.')
        return select_model()
    else:    
        def search_for_model(model):
            try:
                dropdown[1].click()
            except:
                pass
            try:
                model_ = driver.find_element(By.LINK_TEXT, model)
                model_.click()
                print(f"{model} model is selected successfully!")
            except:
                print(f'{model} model does not exist in this list. Please try again.')
                return select_model()
        search_for_model(model)  
        time.sleep(1)

#### After calling select_year_start() function below it will ask you to input the starting year of a car you want to search for and then it will select it. If that make will not exit it will tell you to put the existing one. Call the select_year_start function after calling select_make() and select_model() functions

In [7]:
def select_year_start():
    def scrape_years():
        global years
        years_dropdown = driver.find_element(By.XPATH, '//div[@class = "filter_right"]')
        years = years_dropdown.find_elements(By.XPATH, ('./select'))
    scrape_years()
    start = str(input("Type in the starting year of the car:  "))
    def year_start(start):   
        year_start = years[0]
        try:
            year_start = year_start.find_elements(By.XPATH, ("./option"))
            for year in year_start:
                if year.text == start:
                    print(f"{start} year is selected successfully!")
                    return year.click()
            raise print(f"{start} year does not exist. Please try again.")
        except:
            return select_year_start()
    year_start(start)
    time.sleep(1)

#### After calling select_year_to() function it will ask you to input the ending year of a car you want to search for and then it will select it. If that make will not exit it will tell you to put the existing one.  Call the select_year_start function after calling select_make() and select_model() functions

In [8]:
def select_year_to():
    def scrape_years():
        global years
        years_dropdown = driver.find_element(By.XPATH, '//div[@class = "filter_right"]')
        years = years_dropdown.find_elements(By.XPATH, ('./select'))    
    scrape_years()
    to = str(input("Type in the ending year of the car:  "))
    def year_end(year_to):   
        year_to = years[1]
        try:
            year_to = year_to.find_elements(By.XPATH, ("./option"))
            for year in year_to:
                if year.text == to:
                    print(f"{to} year is selected successfully!")
                    return year.click()
            raise print(f"{to} year does not exist. Please try again.")
        except:
            return select_year_to()
    year_end(to)
    time.sleep(1)
    

#### scrape_page function is called inside create_df and add_next_pages functions. It scrapes all the useful information from every car and adds to lists.

In [9]:
def scrape_page():
    global data
        
    lot_number = []; date_of_sale = []; year = []; vin = []
    condition = []; engine = []; mileage = []; documents = []
    location = []; primary_damage = []; secondary_damage = []
    estimated_retail_value = [];estimated_repair_cost = []
    transmission = []; body_color = [];drive = []; fuel = []
    keys = []; notes = []; avg_price = []; vehicles = []
    price = []; market = []; seller = []
    
    def scrape_data():
        global info, info_p, auction, columns, data
        info = driver.find_elements(By.XPATH, '//span[@class = "blackfont"]')
        info_p = driver.find_element(By.XPATH, '//span[@class = "prices"]')
        auction = driver.find_element(By.XPATH, '//p[@class ="short-story"]/child::span')   


        data = [vehicles, avg_price, lot_number, date_of_sale, 
                year, vin, condition, engine, mileage,  
                documents, location, primary_damage, 
                secondary_damage, estimated_retail_value, 
                estimated_repair_cost, transmission, body_color,
                drive, fuel, keys, notes, seller, price, market] 

        columns = ['vehicles', 'avg_price', 'lot_number', 'date_of_sale', 
                   'year', 'vin', 'condition', 'engine', 'mileage', 
                   'documents', 'location', 'primary_damage', 'secondary_damage', 
                   'estimated_retail_value', 'estimated_repair_cost', 
                   'transmission', 'body_color','drive', 'fuel', 'keys', 
                   'notes','seller', 'price', 'market']   
    
    def adding_data_to_lists():
        scrape_data()
        price.append(info_p.text)
        market.append(auction.text)
        count_ = 0
        if auction.text == ' Copart ':
            data[21].append(info[9].text)
            for i in (info[:9]+info[10:22]):
                data[count_].append(i.text)
                count_ += 1  
        else:
            seller_ = driver.find_element(By.XPATH, '//p[@class = "iaaiseller"]')
            data[21].append(seller_.text)
            for i in info[:21]:
                data[count_].append(i.text)
                count_ += 1

    def go_in_car():
        driver.current_window_handle
        sold_cars_count = driver.find_elements(By.XPATH, '//img[@src = "https://bidfax.info/templates/ru/dleimages/sold2.png"]/../preceding-sibling::a')
        for i in range(len(sold_cars_count) - 1):
            sold_cars = driver.find_elements(By.XPATH, '//img[@src = "https://bidfax.info/templates/ru/dleimages/sold2.png"]/../preceding-sibling::a')
            sold_cars[i].click()
            time.sleep(3)
            adding_data_to_lists()
            driver.current_window_handle
            driver.back()
            time.sleep(3)
            
    go_in_car()


#### create_df function creates dictionary from the lists created in scrape_page function above and turns it into a data frame.

In [10]:
def create_df():
    driver.refresh()
    scrape_page()
    df_data = {}
    for i, j in zip(columns, data):
        df_data[i]= j
    global df
    df = pd.DataFrame(df_data)
    time.sleep(10)


#### add_next_pages function is working only after create_df. It takes the number of pages you want to scrape after the first one and starts scraping them. It creates dataframe like the create_df function does and then appends it into the dataframe created in the create_df function.

In [11]:
def add_next_pages(count):
    def next_page():
        next_page = driver.find_element(By.LINK_TEXT, '»')
        next_page.click()
        time.sleep(10)
    def add_to_df():
        scrape_page()
        df_data = {}
        global df
        for i, j in zip(columns, data):
            df_data[i] = j
        df1 = pd.DataFrame(df_data)
        df = df.append(df1)
        df = df.reset_index(drop = True)
    for i in range(int(count)):
        next_page()
        add_to_df()

#### Below you can see the resulting 6 functions that you need to do a search and scape information about your selected car.

In [12]:
select_make()

 make does not exist in this list. Please try again.
aaa make does not exist in this list. Please try again.
Volkswagen make is selected successfully!


In [13]:
select_model()

 model does not exist in this list. Please try again.
aaa model does not exist in this list. Please try again.
Gti model is selected successfully!


In [14]:
select_year_start()

 year does not exist. Please try again.
aaa year does not exist. Please try again.
2017 year is selected successfully!


In [15]:
select_year_to()

 year does not exist. Please try again.
aaa year does not exist. Please try again.
2020 year is selected successfully!


In [16]:
create_df()

In [17]:
add_next_pages(3)

/var/folders/7_/hrpf2wd52k91vfg1qd2m7ch80000gn/T/ipykernel_95020/1723988589.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df1)


In [20]:
driver.close()

#### Below you can see resulting information about Volkswagen Gti car from 2017 to 2020 within first 4 pages.

In [23]:
df

,vehicles,avg_price,lot_number,date_of_sale,year,vin,condition,engine,mileage,documents,...,estimated_repair_cost,transmission,body_color,drive,fuel,keys,notes,seller,price,market
0,58,$6541,71574332,15.03.2023,2018,3VW447AU1JM260953,Run And Drive,2 L 4,90533 miles (Actual),Tx - Salvage Vehicle Title,...,20128,Automatic,Black,Front-Wheel Drive,Gas,Yes,Not specified,Not specified,7400,Copart
1,38,$7670,41964373,03.03.2023,2020,3VW5T7AUXLM000819,Run And Drive,2.0L 4,18720 miles (Actual),Pa - Reconstructed Clean,...,0,Manual,Charcoal,Front-Wheel Drive,Gas,Yes,Not specified,Not specified,11500,Copart
2,175,$4584,41350703,15.03.2023,2017,3VW547AU3HM072191,Not specified,2 L 4,60517 miles (Actual),Wi - Cert Of Title-Salvage Vehicle,...,12612,Manual,Blue,Front-Wheel Drive,Gas,No,Not specified,Not specified,5700,Copart
3,51,$7225,40053243,14.03.2023,2019,3VW6T7AUXKM002086,Run And Drive,2.0L 4,31514 miles (Actual),Fl - Certificate Of Destruction,...,27572,Automatic,White,Front-Wheel Drive,Gas,Yes,Not specified,Not specified,7300,Copart
4,58,$6541,36728403,14.03.2023,2018,3VW547AU7JM296182,Not specified,2 L 4,0 miles (Not Actual),Md - Cert Of Salvage > 75% Damage,...,21487,Manual,White,Front-Wheel Drive,Gas,Yes,Not specified,Not specified,625,Copart
5,175,$4584,72061822,10.03.2023,2017,3VW447AU9HM042768,Enhanced Vehicles,2 L 4,57229 miles (Actual),Ct - Cert Of Title-Salvage,...,16992,Automatic,White,Front-Wheel Drive,Gas,No,Not specified,Not specified,4300,Copart
6,58,$6541,61368442,10.03.2023,2018,3VW547AU2JM250937,Enhanced Vehicles,2 L 4,39693 miles (Actual),Mi - Cert Of Title-Salvage Vehicle,...,19654,Manual,Black,Front-Wheel Drive,Gas,Yes,Not specified,Not specified,4150,Copart
7,51,$7225,41979843,03.03.2023,2019,3VW6T7AU3KM008537,Run And Drive,2.0L 4,43428 miles (Actual),Fl - Cert Of Title-Rebuilt,...,0,Automatic,Gray,Front-Wheel Drive,Gas,Yes,Not specified,Not specified,6900,Copart
8,175,$4584,41896023,10.03.2023,2017,3VW547AU8HM042149,Enhanced Vehicles,2 L 4,175203 miles (Actual),Fl - Certificate Of Destruction,...,13656,Manual,Black,Front-Wheel Drive,Gas,Yes,Not specified,Not specified,2250,Copart
9,175,$4584,41106053,10.03.2023,2017,3VW447AU7HM034362,Enhanced Vehicles,2 L 4,0 miles (Not Actual),Mo - Salvage Certificate Of Title,...,13684,Automatic,Black,Front-Wheel Drive,Gas,Yes,Not specified,Not specified,4150,Copart


#### Code below will create a csv file with the data above. You can put the path for storing csv file in the brackets inside the parenthesis of the method below.

In [ ]:
df.to_csv()